In [2]:
import numpy as np
import cv2
import time
import pyautogui
from directkeys import PressKey,ReleaseKey, W, A, S, D
from draw_lanes import draw_lanes
from grabscreen import grab_screen
from getkeys import key_check
from create_training_data import keys_to_output
import os


file_name = 'training_data.npy'

if os.path.isfile(file_name):
    print('File exists, loading previous data!')
    training_data = list(np.load(file_name))
else:
    print('File does not exist, starting fresh!')
    training_data = []

def straight():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)

def left():
    PressKey(A)
    ReleaseKey(W)
    ReleaseKey(D)
    ReleaseKey(A)

def right():
    PressKey(D)
    ReleaseKey(A)
    ReleaseKey(W)
    ReleaseKey(D)

def slow_ya_roll():
    ReleaseKey(W)
    ReleaseKey(A)
    ReleaseKey(D)


    
def process_img(image):
    original_image = image
    # convert to gray
    processed_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # edge detection
    processed_img =  cv2.Canny(processed_img, threshold1 = 500, threshold2=200)
    processed_img = cv2.GaussianBlur(processed_img,(5,5),0)
    lines = cv2.HoughLinesP(processed_img, 1, np.pi/180, 180, np.array([]) ,    20,         15)
    m1=0
    m2=0
    
    try:
        l1, l2, m1, m2 = draw_lanes(original_image,lines)
        cv2.line(original_image, (l1[0], l1[1]), (l1[2], l1[3]), [0,255,0], 30)
        cv2.line(original_image, (l2[0], l2[1]), (l2[2], l2[3]), [0,255,0], 30)
    except Exception as e:
        print(str(e))
        pass
    try:
        for coords in lines:
            coords = coords[0]
            try:
                cv2.line(processed_img, (coords[0], coords[1]), (coords[2], coords[3]), [255,0,0], 3)
                
                
            except Exception as e:
                print(str(e))
    except Exception as e:
        pass
    return processed_img,original_image,m1,m2


def main():
    for i in list(range(4))[::-1]:
        print(i+1)
        time.sleep(1)


    while(True):
        screen =  grab_screen(region=(0,40, 1024, 800))
        last_time = time.time()
        screen = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)
        screen = cv2.resize(screen, (80,60))
        keys = key_check()
        output = keys_to_output(keys)
        training_data.append([screen,output])
       
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
        if len(training_data) % 500 == 0:
            print(len(training_data))
            np.save(file_name,training_data)
#main()
#main
if __name__=='__main__':
    main()

File exists, loading previous data!
4
3
2
1
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500


KeyboardInterrupt: 

In [5]:
#balance_data
# coding: utf-8

# In[1]:


import numpy as np
import pandas as pd
from collections import Counter
from random import shuffle

train_data = np.load('training_data.npy')

df = pd.DataFrame(train_data)
print(df.head())
print(Counter(df[1].apply(str)))

lefts = []
rights = []
forwards = []

shuffle(train_data)

for data in train_data:
    img = data[0]
    choice = data[1]

    if choice == [1,0,0]:
        lefts.append([img,choice])
    elif choice == [0,1,0]:
        forwards.append([img,choice])
    elif choice == [0,0,1]:
        rights.append([img,choice])
    else:
        print('no matches')


forwards = forwards[:len(lefts)][:len(rights)]
lefts = lefts[:len(forwards)]
rights = rights[:len(forwards)]
final_data = forwards + lefts + rights
shuffle(final_data)

np.save('training_data_v2.npy', final_data)



                                                   0          1
0  [[102, 85, 83, 89, 89, 95, 105, 95, 68, 75, 94...  [0, 1, 0]
1  [[111, 93, 91, 97, 97, 104, 115, 104, 74, 82, ...  [0, 1, 0]
2  [[119, 100, 97, 104, 104, 112, 123, 111, 79, 8...  [0, 1, 0]
3  [[124, 104, 102, 109, 109, 117, 129, 117, 83, ...  [0, 1, 0]
4  [[128, 108, 105, 112, 112, 120, 132, 120, 86, ...  [0, 1, 0]
Counter({'[0, 1, 0]': 28591, '[0, 0, 1]': 4151, '[1, 0, 0]': 3758})
